### These files are not in any standard image format. You have to write your own (very simple) program to read them.

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import tempfile

In [4]:
import numpy as np
import tensorflow as tf
from six.moves import urllib
from six.moves import xrange
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

In [5]:
mnist = read_data_sets('data',one_hot = True)

Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz


In [5]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000000006021668>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000000000ACA4588>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000000052B7588>)

In [16]:
#Session是与C++的连接，而计算图存在于Session中
sess = tf.InteractiveSession()

### 矩阵乘法等耗时操作，在Python环境外来计算，Python的作用，就是**构建可以在外部运行的计算图**

In [6]:
#占位符：在TensorFlow运行某一计算时，根据该占位符输入具体值
#None指batch，784指x的dimension
x = tf.placeholder(tf.float32,shape = [None,784])
y_ = tf.placeholder(tf.float32,shape = [None,10])
#模型参数一般用Variable来表示
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [16]:
#Variable需要在session初始化后，才能在session中使用。
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [17]:
#交叉熵：衡量目标和预测误差的一种方法，即损失函数
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

In [18]:
#往计算图添加个新操作（包括，计算梯度，计算步长变化，更新参数）
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [19]:
#50报错,是因为在input_data时没有加one-hot,所以batch_y显示的为1到9的数值。
for i in range(1000):
    batch_x, batch_y = mnist.train.next_batch(100)
    train_step.run(feed_dict={x:batch_x,y_:batch_y})

In [20]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [21]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [22]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9047


## 多层卷积神经网络

In [7]:
#Initialized
def weight_variable(shape):
    #权重在初始化时加入少量噪声stddev = 0.1以打破对称性，避免0梯度
    initial = tf.truncated_normal(shape,stddev= 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    #用较小正数0.1来初始化bias，避免出现dead neurons
    initial = tf.constant(0.1, shape =shape)
    return tf.Variable(initial)

In [8]:
#定义CONV和Max_pool
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides= [1,1,1,1], padding= 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize= [1,2,2,1],strides = [1,2,2,1],padding='SAME')

In [9]:
#-1意思是数量未知
x_image = tf.reshape(x, [-1,28,28,1])

In [10]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [11]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [12]:
w_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
#使用计算是tf.matmul而不是tf.conv2d
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

In [13]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [14]:
w_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

In [17]:
cross_entroy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entroy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
sess.run(tf.global_variables_initializer())

In [54]:
for i in range(5000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1],keep_prob:1.0})
        #%g是什么,为什么还是y_
        print('step %d,training accuracy is %g'%(i,train_accuracy))
        
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
    


step 0,training accuracy is 0.96
step 100,training accuracy is 0.96
step 200,training accuracy is 1
step 300,training accuracy is 0.98
step 400,training accuracy is 0.98
step 500,training accuracy is 0.98
step 600,training accuracy is 0.94
step 700,training accuracy is 1
step 800,training accuracy is 0.98
step 900,training accuracy is 1
step 1000,training accuracy is 0.96
step 1100,training accuracy is 0.98
step 1200,training accuracy is 0.96
step 1300,training accuracy is 1
step 1400,training accuracy is 1
step 1500,training accuracy is 0.98
step 1600,training accuracy is 0.96
step 1700,training accuracy is 0.98
step 1800,training accuracy is 1
step 1900,training accuracy is 1
step 2000,training accuracy is 1
step 2100,training accuracy is 0.98
step 2200,training accuracy is 0.98
step 2300,training accuracy is 0.98
step 2400,training accuracy is 0.98
step 2500,training accuracy is 0.98
step 2600,training accuracy is 1
step 2700,training accuracy is 1
step 2800,training accuracy is 0.9

In [55]:
#电脑算力不够，只能测试3500个样本
test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images[:3500,:], y_: mnist.test.labels[:3500,:], keep_prob: 1.0})

In [56]:
print('test_accuracy is %g'% test_accuracy)

test_accuracy is 0.984286
